**Problem Definition**

Classification of the weather image into one of the following classes

1. Sunrise
2. Shine
3. Rain
4. Cloudy

**Data**

[Dataset](https://www.kaggle.com/datasets/pratik2901/multiclass-weather-dataset)

1. The dataset contains images of corresponding to the Sunrise, Shine, Rain, Cloudy weather conditions
2. Class Distribution
    1. Sunrise - 357 
    2. Shine - 253
    3. Rain - 215
    4. Clody - 300
3. The updated folder structure of the dataset facilitate the data load prodecure

Data Publication
1. The dataset was published on [Mendeley Data](https://data.mendeley.com/datasets/4drtyfjtfy/1)
2. Cite: Ajayi, Gbeminiyi (2018), Multi-class Weather Dataset for Image Classification, Mendeley Data, v1
3. [DOI](http://dx.doi.org/10.17632/4drtyfjtfy.1)
4. Published: 2018-09-13
5. Institutions: University of South Africa - Science Campus
6. Licence: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)


**Evaluation**

Accuracy

**Modelling**

Fully Convolutional Network (FCN)